###
已知两类样本的数据如下： 
 
 
试用Fisher判别函数法，求出最佳投影方向W，及分类阈值y0

思路：
令J(w)最大


In [1]:
import numpy as np

### 对数据的理解
首先w1与w2分别为两个类别的数据  
这个例子中，一个数据有两个维度，则需要将这两个维度投影到易分离两个类别的新空间   
通过fisher算法，可以表征类间差别和类内相似，进而找到最佳投影方向！

In [2]:
w1=[(5,37),(7,30),(10,35),(11.5,40),(14,38),(12,31)]
w2=[(35,21.5),(39,21.7),(34,16),(37,17)]

In [4]:
w1=np.asarray(w1)
w2=np.asarray(w2)

In [5]:
w1

array([[ 5. , 37. ],
       [ 7. , 30. ],
       [10. , 35. ],
       [11.5, 40. ],
       [14. , 38. ],
       [12. , 31. ]])

由于每一行数据是单个数据结果，不利于应用平均函数
我们可以利用numpy的swapaxes交换行列，在只有两维时等价于
``` python
X0=w1[:,0]
X1=w1[:,1]

```

In [9]:
X1=np.swapaxes(w1,0,1)
X2=np.swapaxes(w2,0,1)

计算交换行列后的平均数，其实利用numpy的函数也可以轻松计算得出按列求平均的结果，即`axis=0`

In [14]:
# compute mean for each row
print(X1.mean(axis=1))
Miu1=X1.mean(axis=1)
Miu2=X2.mean(axis=1)
print(np.cov(X1))

[ 9.91666667 35.16666667]
[[11.24166667  3.31666667]
 [ 3.31666667 15.76666667]]


In [28]:
# 看一下原始数据减去平均值的结果，numpy就是正常的矩阵减法：原数据的每一行都减去平均值，符合需要
w1-Miu1

array([[-4.91666667,  1.83333333],
       [-2.91666667, -5.16666667],
       [ 0.08333333, -0.16666667],
       [ 1.58333333,  4.83333333],
       [ 4.08333333,  2.83333333],
       [ 2.08333333, -4.16666667]])

### 协方差covariance
使用numpy内置函数cov()与ppt上的公式计算结果不同，因为标准的协方差公式为  
$$cov=\frac{1}{n-1} lazyCov$$
$$lazyCov=(X-\mu)(X-\mu)^{T}$$

In [23]:
def lazyCov(X,miu):
    return np.dot(np.transpose(X-miu),X-miu)

In [30]:
Sigma1=lazyCov(w1,Miu1)
Sigma2=lazyCov(w2,Miu2)

In [25]:
lazyCov(w1,Miu1)/5

array([[11.24166667,  3.31666667],
       [ 3.31666667, 15.76666667]])

In [31]:
Sigma2

array([[14.75,  9.55],
       [ 9.55, 26.53]])

### 类间差别与类内相似

In [36]:
Sb=np.dot(Miu1-Miu2,np.transpose(Miu1-Miu2))
print(Sb)

953.1913888888889


In [50]:
Sb=np.dot(np.transpose(Miu1-Miu2).reshape(2,1),(Miu1-Miu2).reshape(1,2))
print(Sb)
print(Miu1-Miu2)
print(np.transpose(Miu1-Miu2).shape)

[[ 693.44444444 -424.40555556]
 [-424.40555556  259.74694444]]
[-26.33333333  16.11666667]
(2,)


In [35]:
Sw=Sigma1+Sigma2
print(Sw)

[[ 70.95833333  26.13333333]
 [ 26.13333333 105.36333333]]


### 经过一系列推导，可知最佳的projection$w*$由下面的公式决定：
$$w*=(\sigma_{1}+\sigma_{2})^{-1}(\mu_{1}-\mu_{2})$$

In [67]:
projectionW=np.dot((Miu1-Miu2).reshape(1,2),np.linalg.inv(Sigma1+Sigma2))
print(projectionW)

[[-0.47041598  0.26964033]]


In [65]:
Zt=np.dot(projectionW,(Miu1+Miu2).transpose())/2
print(Zt)

[-3.54926888]


In [68]:
Z1=np.dot(projectionW,w1.transpose())
Z2=np.dot(projectionW,w2.transpose())

### 检验分类结果，全部区分开来

In [70]:
print(Z1[Z1>Zt])
print(Z1[Z1<Zt])
print(Z2[Z2>Zt])
print(Z2[Z2<Zt])

[7.62461243 4.79629813 4.73325185 5.37582954 3.66050892 2.71385855]
[]
[]
[-10.6672922  -12.49502806 -11.67989805 -12.82150566]
